In [7]:
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
import json
from pytz import timezone
import time

In [3]:
import requests
import zipfile
import io

# Define the URL and your API key
api_key = 'afec7635-a79b-4ccb-b87b-5c8d9cf5b36c'
operator = 'CT'
url = f'http://api.511.org/transit/datafeeds?api_key={api_key}&operator_id={operator}'

# Download the GTFS zip file
response = requests.get(url)
response.raise_for_status()  # Ensure we got a successful response

# Unzip the file
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall('gtfs_data')  # Extract to a directory named 'gtfs_data'

print("GTFS data downloaded and unzipped successfully.")


GTFS data downloaded and unzipped successfully.


In [3]:
stops_df = pd.read_csv('gtfs_data/stops.txt')

In [4]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,stop_desc,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,platform_code
0,22nd_street,22nd_street,22nd Street,37.756972,-122.392492,74633,NaN,NaN,1,NaN,America/Los_Angeles,0,NaN
1,70021,70021,22nd Street Caltrain Station Northbound,37.757599,-122.391880,74634,NaN,NaN,0,22nd_street,NaN,2,NaN
2,70022,70022,22nd Street Caltrain Station Southbound,37.757583,-122.392404,74634,NaN,NaN,0,22nd_street,NaN,2,NaN
3,bayshore,bayshore,Bayshore,37.707660,-122.401665,74633,NaN,https://www.caltrain.com/station/bayshore,1,NaN,America/Los_Angeles,0,NaN
4,70031,70031,Bayshore Caltrain Station Northbound,37.709537,-122.401586,74634,NaN,NaN,0,bayshore,NaN,1,NaN


In [5]:
# drop all non-numeric rows from stop_id
stops_df = stops_df[stops_df['stop_id'].str.isnumeric()]
stops_df.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,stop_desc,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,platform_code
1,70021,70021,22nd Street Caltrain Station Northbound,37.757599,-122.391880,74634,NaN,NaN,0,22nd_street,NaN,2,NaN
2,70022,70022,22nd Street Caltrain Station Southbound,37.757583,-122.392404,74634,NaN,NaN,0,22nd_street,NaN,2,NaN
4,70031,70031,Bayshore Caltrain Station Northbound,37.709537,-122.401586,74634,NaN,NaN,0,bayshore,NaN,1,NaN
5,70032,70032,Bayshore Caltrain Station Southbound,37.709544,-122.401980,74634,NaN,NaN,0,bayshore,NaN,1,NaN
7,70121,70121,Belmont Caltrain Station Northbound,37.520890,-122.275738,74635,NaN,NaN,0,belmont,NaN,1,NaN


In [45]:
url = f'https://api.511.org/transit/VehicleMonitoring?api_key={api_key}&agency={operator}'
# Fetch the GTFS-RT data
response = requests.get(url)
response.raise_for_status()  #

In [46]:
response.content

b'\xef\xbb\xbf{"Siri":{"ServiceDelivery":{"ResponseTimestamp":"2024-07-07T20:50:57Z","ProducerRef":"CT","Status":true,"VehicleMonitoringDelivery":{"version":"1.4","ResponseTimestamp":"2024-07-07T20:50:57Z","VehicleActivity":[{"RecordedAtTime":"2024-07-07T20:50:42Z","ValidUntilTime":"","MonitoredVehicleJourney":{"LineRef":"L2","DirectionRef":"N","FramedVehicleJourneyRef":{"DataFrameRef":"2024-07-07","DatedVehicleJourneyRef":"237"},"PublishedLineName":"Local","OperatorRef":"CT","OriginRef":"70261","OriginName":"San Jose Diridon Caltrain Station Northbound","DestinationRef":"70011","DestinationName":"San Francisco Caltrain Station Northbound","Monitored":true,"InCongestion":null,"VehicleLocation":{"Longitude":"-122.400963","Latitude":"37.7122917"},"Bearing":null,"Occupancy":null,"VehicleRef":"237","MonitoredCall":{"StopPointRef":"70021","StopPointName":"22nd Street Caltrain Station Northbound","DestinationDisplay":"San Francisco","VehicleLocationAtStop":"","VehicleAtStop":"","AimedArrival

In [49]:
import json

data = response.content  # Your JSON data

# Decode the byte string to a regular string
data_str = data.decode('utf-8-sig')

# Parse the JSON data
json_data = json.loads(data_str)

# Extracting the relevant information
vehicle_activities = json_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']['VehicleActivity']

for activity in vehicle_activities:
    journey = activity['MonitoredVehicleJourney']
    print(f"trip_id: {journey['VehicleRef']}")
    print(f"Line: {journey['PublishedLineName']}")
    print(f"Direction: {journey['DirectionRef']}")
    print(f"Origin: {journey['OriginName']}")
    print(f"Destination: {journey['DestinationName']}")
    print(f"Vehicle Location: Lat {journey['VehicleLocation']['Latitude']}, Lon {journey['VehicleLocation']['Longitude']}")
    
    monitored_call = journey['MonitoredCall']
    print(f"Next Stop: {monitored_call['StopPointName']}")
    print(f"Expected Arrival Time: {monitored_call['ExpectedArrivalTime']}")
    print(f"Expected Departure Time: {monitored_call['ExpectedDepartureTime']}")
    
    onward_calls = journey['OnwardCalls']['OnwardCall']
    for call in onward_calls:
        print(f"  Onward Stop: {call['StopPointName']}")
        print(f"  Expected Arrival Time: {call['ExpectedArrivalTime']}")
        print(f"  Expected Departure Time: {call['ExpectedDepartureTime']}")
    print()


trip_id: 237
Line: Local
Direction: N
Origin: San Jose Diridon Caltrain Station Northbound
Destination: San Francisco Caltrain Station Northbound
Vehicle Location: Lat 37.7122917, Lon -122.400963
Next Stop: 22nd Street Caltrain Station Northbound
Expected Arrival Time: 2024-07-07T20:52:05Z
Expected Departure Time: 2024-07-07T20:53:05Z

trip_id: 240
Line: Local
Direction: S
Origin: San Francisco Caltrain Station Southbound
Destination: Tamien Caltrain Station Southbound
Vehicle Location: Lat 37.5029411, Lon -122.254463
Next Stop: San Carlos Caltrain Station Southbound
Expected Arrival Time: 2024-07-07T20:51:25Z
Expected Departure Time: 2024-07-07T20:52:25Z
  Onward Stop: Redwood City Caltrain Station Southbound
  Expected Arrival Time: 2024-07-07T20:54:12Z
  Expected Departure Time: 2024-07-07T20:55:12Z
  Onward Stop: Menlo Park Caltrain Station Southbound
  Expected Arrival Time: 2024-07-07T20:58:11Z
  Expected Departure Time: 2024-07-07T20:59:11Z
  Onward Stop: Palo Alto Caltrain Stat

In [50]:
json_data

{'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-07-07T20:50:57Z',
   'ProducerRef': 'CT',
   'Status': True,
   'VehicleMonitoringDelivery': {'version': '1.4',
    'ResponseTimestamp': '2024-07-07T20:50:57Z',
    'VehicleActivity': [{'RecordedAtTime': '2024-07-07T20:50:42Z',
      'ValidUntilTime': '',
      'MonitoredVehicleJourney': {'LineRef': 'L2',
       'DirectionRef': 'N',
       'FramedVehicleJourneyRef': {'DataFrameRef': '2024-07-07',
        'DatedVehicleJourneyRef': '237'},
       'PublishedLineName': 'Local',
       'OperatorRef': 'CT',
       'OriginRef': '70261',
       'OriginName': 'San Jose Diridon Caltrain Station Northbound',
       'DestinationRef': '70011',
       'DestinationName': 'San Francisco Caltrain Station Northbound',
       'Monitored': True,
       'InCongestion': None,
       'VehicleLocation': {'Longitude': '-122.400963',
        'Latitude': '37.7122917'},
       'Bearing': None,
       'Occupancy': None,
       'VehicleRef': '237',
       'Mo

In [48]:
journey['VehicleLocation']['Latitude']

'37.4544983'

In [9]:
# get list of unique trip_ids
trip_ids = set()
for activity in vehicle_activities:
    trip_ids.add(activity['MonitoredVehicleJourney']['VehicleRef'])
print(trip_ids)

{'142', '518', '138', '139', '141', '137', '134'}


In [10]:
import math

def haversine(lat1, lon1, lat2, lon2):
       
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371000  # Radius of Earth in meters
    distance = r * c
    
    return distance
def has_train_arrived(train_lat, train_lon, stop_lat, stop_lon, threshold=100):
    # Calculate the distance between the train and the stop
    distance = haversine(train_lat, train_lon, stop_lat, stop_lon)
    
    # Check if the distance is within the defined threshold
    if distance <= threshold:
        return True
    else:
        return False



In [15]:
# for each trip_id, get the next stop
data =[]
# get list of unique trip_ids
trip_ids = set()
for activity in vehicle_activities:
    trip_ids.add(activity['MonitoredVehicleJourney']['VehicleRef'])
for trip_id in trip_ids:
    for activity in vehicle_activities:
        journey = activity['MonitoredVehicleJourney']
        if journey['VehicleRef'] == trip_id:
            print(f"trip_id: {journey['VehicleRef']}")
            vehicle_lat = float(journey['VehicleLocation']['Latitude'])
            vehicle_lon = float(journey['VehicleLocation']['Longitude'])
            print(f"Vehicle Location: Lat {vehicle_lat}, Lon {vehicle_lon}")
    
            monitored_call = journey['MonitoredCall']
            stop_id = monitored_call['StopPointRef']
            print(f"Next Stop: {monitored_call['StopPointName']}, Stop ID: {stop_id}")
            # get stop location
            stop = stops_df[stops_df['stop_id'] == stop_id]
            stop_lat = float(stop['stop_lat'].values[0])
            stop_lon = float(stop['stop_lon'].values[0])
            print(f"Stop Location: Lat {stop_lat}, Lon {stop_lon}")
            print(f"Distance to Stop: {haversine(vehicle_lat, vehicle_lon, stop_lat, stop_lon):.2f} meters")
            has_arrived = has_train_arrived(vehicle_lat, vehicle_lon, stop_lat, stop_lon)
            if has_arrived:
                print(f"Train has arrived")
                expected_arrival_time = monitored_call['ExpectedArrivalTime']
                actual_arrival_time = datetime.now()
                expected_arrival_time = datetime.strptime(expected_arrival_time, '%Y-%m-%dT%H:%M:%S%z')  - timedelta(hours=7)
                expected_arrival_time = expected_arrival_time.replace(tzinfo=None)
                late = actual_arrival_time > expected_arrival_time + timedelta(minutes=1)
                mins_late = (actual_arrival_time - expected_arrival_time).seconds // 60
                data.append({'trip_id': trip_id, 'stop_id': stop_id, 'stop_name': monitored_call['StopPointName'], 'expected_arrival_time': expected_arrival_time, 'actual_arrival_time': actual_arrival_time, 'late': late, 'mins_late': mins_late})
                break

            break

trip_id: 228
Vehicle Location: Lat 37.4544601, Lon -122.181839
Next Stop: Palo Alto Caltrain Station Southbound, Stop ID: 70172
Stop Location: Lat 37.443405, Lon -122.164697
Distance to Stop: 1949.62 meters
trip_id: 232
Vehicle Location: Lat 37.7754288, Lon -122.395782
Next Stop: 22nd Street Caltrain Station Southbound, Stop ID: 70022
Stop Location: Lat 37.757583, Lon -122.392404
Distance to Stop: 2006.45 meters


In [17]:
stop_id,trip_id

('70022', '232')

In [31]:
stop_times = pd.read_csv('gtfs_data/stop_times.txt')
arrival_time = stop_times.loc[(stop_times.trip_id==int(trip_id)) & (stop_times.stop_id==int(stop_id)),'arrival_time'].values[0]
arrival_time
# datetime.strptime(arrival_time, '%H:%M:%S')

'11:03:00'

In [33]:
from datetime import datetime, time, timedelta

def convert_schedule_time(time_str, date):
    # Parse the time string
    hours, minutes, seconds = map(int, time_str.split(':'))
    
    # Handle times past midnight
    if hours >= 24:
        hours -= 24
        date += timedelta(days=1)
    
    # Create a time object
    time_obj = time(hour=hours, minute=minutes, second=seconds)
    
    # Combine the date and time
    return datetime.combine(date, time_obj)

In [34]:
current_time = datetime.now()
current_date = current_time.date()
scheduled_arrival_time = convert_schedule_time(arrival_time, current_date)

In [35]:
scheduled_arrival_time

datetime.datetime(2024, 7, 6, 11, 3)

In [44]:
(current_time - scheduled_arrival_time).seconds//60

218

In [12]:
# logic overview
# Get a list of the current trains running
# For each train, get the next stop
# calculate the distance between the train and the stop using the lat/long of the train from the gtfs-rt feed and the lat/long of the stop from gtfs_data/stops.txt
# If the train is within 500 meters of the stop, record the train, stop, date, and time

In [13]:
pd.DataFrame(data)

""


In [8]:
import math

def haversine(lat1, lon1, lat2, lon2):
       
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371000  # Radius of Earth in meters
    distance = r * c
    
    return distance
def has_train_arrived(train_lat, train_lon, stop_lat, stop_lon, threshold=100):
    # Calculate the distance between the train and the stop
    distance = haversine(train_lat, train_lon, stop_lat, stop_lon)
    
    # Check if the distance is within the defined threshold
    if distance <= threshold:
        return True
    else:
        return False
url = f'https://api.511.org/transit/VehicleMonitoring?api_key={api_key}&agency={operator}'
stop_data =[]
time_to_run = 300
start_time = time.time()
while time.time() - start_time < time_to_run:
    # Fetch the GTFS-RT data
    response = requests.get(url)
    response.raise_for_status()  #
    # for each trip_id, get the next stop
    data = response.content  # Your JSON data

    # Decode the byte string to a regular string
    data_str = data.decode('utf-8-sig')

    # Parse the JSON data
    json_data = json.loads(data_str)

    # Extracting the relevant information
    vehicle_activities = json_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']['VehicleActivity']

    # get list of unique trip_ids
    trip_ids = set()
    for activity in vehicle_activities:
        trip_ids.add(activity['MonitoredVehicleJourney']['VehicleRef'])
    for trip_id in trip_ids:
        for activity in vehicle_activities:
            journey = activity['MonitoredVehicleJourney']
            if journey['VehicleRef'] == trip_id:
                # print(f"trip_id: {journey['VehicleRef']}")
                vehicle_lat = float(journey['VehicleLocation']['Latitude'])
                vehicle_lon = float(journey['VehicleLocation']['Longitude'])
                # print(f"Vehicle Location: Lat {vehicle_lat}, Lon {vehicle_lon}")
        
                monitored_call = journey['MonitoredCall']
                stop_id = monitored_call['StopPointRef']
                # print(f"Next Stop: {monitored_call['StopPointName']}, Stop ID: {stop_id}")
                # get stop location
                stop = stops_df[stops_df['stop_id'] == stop_id]
                stop_lat = float(stop['stop_lat'].values[0])
                stop_lon = float(stop['stop_lon'].values[0])
                # print(f"Stop Location: Lat {stop_lat}, Lon {stop_lon}")
                # print(f"Distance to Stop: {haversine(vehicle_lat, vehicle_lon, stop_lat, stop_lon):.2f} meters")
                has_arrived = has_train_arrived(vehicle_lat, vehicle_lon, stop_lat, stop_lon)
                if has_arrived:
                    print(f"Train has arrived")
                    expected_arrival_time = monitored_call['ExpectedArrivalTime']
                    actual_arrival_time = datetime.now()
                    expected_arrival_time = datetime.strptime(expected_arrival_time, '%Y-%m-%dT%H:%M:%S%z')  - timedelta(hours=7)
                    expected_arrival_time = expected_arrival_time.replace(tzinfo=None)
                    late = actual_arrival_time > expected_arrival_time + timedelta(minutes=1)
                    mins_late = (actual_arrival_time - expected_arrival_time).seconds // 60
                    stop_data.append({'trip_id': trip_id, 'stop_id': stop_id, 'stop_name': monitored_call['StopPointName'], 'expected_arrival_time': expected_arrival_time, 'actual_arrival_time': actual_arrival_time, 'late': late, 'mins_late': mins_late})
                    break

                break
    time.sleep(60)

NameError: name 'stops_df' is not defined

In [19]:
df = pd.DataFrame(stop_data)

In [20]:
df

,trip_id,stop_id,stop_name,expected_arrival_time,actual_arrival_time,late,mins_late
0,138,70052,San Bruno Caltrain Station Southbound,2024-07-05 21:37:51,2024-07-05 21:37:38.200495,False,1439
1,134,70242,Santa Clara Caltrain Station Southbound,2024-07-05 21:52:29,2024-07-05 21:54:39.731727,True,2
2,138,70122,Belmont Caltrain Station Southbound,2024-07-05 21:57:39,2024-07-05 21:59:40.044497,True,2
3,139,70131,San Carlos Caltrain Station Northbound,2024-07-05 21:59:09,2024-07-05 22:00:40.114078,True,1
4,134,70262,San Jose Diridon Caltrain Station Southbound,2024-07-05 21:59:39,2024-07-05 22:01:40.171392,True,2
5,138,70132,San Carlos Caltrain Station Southbound,2024-07-05 22:03:20,2024-07-05 22:02:40.243546,False,1439
6,134,70262,San Jose Diridon Caltrain Station Southbound,2024-07-05 21:59:39,2024-07-05 22:02:40.245549,True,3
7,138,70132,San Carlos Caltrain Station Southbound,2024-07-05 22:03:20,2024-07-05 22:03:40.287727,False,0
8,134,70262,San Jose Diridon Caltrain Station Southbound,2024-07-05 21:59:39,2024-07-05 22:03:40.289727,True,4
9,138,70132,San Carlos Caltrain Station Southbound,2024-07-05 22:03:20,2024-07-05 22:04:40.347425,True,1


In [11]:
# Define the URL and your API key
api_key = 'afec7635-a79b-4ccb-b87b-5c8d9cf5b36c'
operator = 'CT'
url = f'http://api.511.org/transit/datafeeds?api_key={api_key}&operator_id={operator}'

# Download the GTFS zip file
response = requests.get(url)
response.raise_for_status()  # Ensure we got a successful response

# Unzip the file
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall('gtfs_data')  # Extract to a directory named 'gtfs_data'

print("GTFS data downloaded and unzipped successfully.")
stops_df = pd.read_csv('gtfs_data/stops.txt')
# drop all non-numeric rows from stop_id
stops_df = stops_df[stops_df['stop_id'].str.isnumeric()]
import math

def haversine(lat1, lon1, lat2, lon2):
       
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371000  # Radius of Earth in meters
    distance = r * c
    
    return distance
def has_train_arrived(train_lat, train_lon, stop_lat, stop_lon, threshold=100):
    # Calculate the distance between the train and the stop
    distance = haversine(train_lat, train_lon, stop_lat, stop_lon)
    
    # Check if the distance is within the defined threshold
    if distance <= threshold:
        return True
    else:
        return False
url = f'https://api.511.org/transit/VehicleMonitoring?api_key={api_key}&agency={operator}'
stop_data =[]
time_to_run = 300
start_time = time.time()
while time.time() - start_time < time_to_run:
    # Fetch the GTFS-RT data
    response = requests.get(url)
    response.raise_for_status()  #
    # for each trip_id, get the next stop
    data = response.content  # Your JSON data

    # Decode the byte string to a regular string
    data_str = data.decode('utf-8-sig')

    # Parse the JSON data
    json_data = json.loads(data_str)

    # Extracting the relevant information
    vehicle_activities = json_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']['VehicleActivity']

    # get list of unique trip_ids
    trip_ids = set()
    for activity in vehicle_activities:
        trip_ids.add(activity['MonitoredVehicleJourney']['VehicleRef'])
    for trip_id in trip_ids:
        for activity in vehicle_activities:
            journey = activity['MonitoredVehicleJourney']
            if journey['VehicleRef'] == trip_id:
                # print(f"trip_id: {journey['VehicleRef']}")
                vehicle_lat = float(journey['VehicleLocation']['Latitude'])
                vehicle_lon = float(journey['VehicleLocation']['Longitude'])
                # print(f"Vehicle Location: Lat {vehicle_lat}, Lon {vehicle_lon}")
        
                monitored_call = journey['MonitoredCall']
                stop_id = monitored_call['StopPointRef']
                # print(f"Next Stop: {monitored_call['StopPointName']}, Stop ID: {stop_id}")
                # get stop location
                stop = stops_df[stops_df['stop_id'] == stop_id]
                stop_lat = float(stop['stop_lat'].values[0])
                stop_lon = float(stop['stop_lon'].values[0])
                # print(f"Stop Location: Lat {stop_lat}, Lon {stop_lon}")
                # print(f"Distance to Stop: {haversine(vehicle_lat, vehicle_lon, stop_lat, stop_lon):.2f} meters")
                has_arrived = has_train_arrived(vehicle_lat, vehicle_lon, stop_lat, stop_lon)
                if has_arrived:
                    print(f"Train has arrived")
                    expected_arrival_time = monitored_call['ExpectedArrivalTime']
                    actual_arrival_time = datetime.now()
                    expected_arrival_time = datetime.strptime(expected_arrival_time, '%Y-%m-%dT%H:%M:%S%z')  - timedelta(hours=7)
                    expected_arrival_time = expected_arrival_time.replace(tzinfo=None)
                    late = actual_arrival_time > expected_arrival_time + timedelta(minutes=1)
                    mins_late = (actual_arrival_time - expected_arrival_time).seconds // 60
                    stop_data.append({'trip_id': trip_id, 'stop_id': stop_id, 'stop_name': monitored_call['StopPointName'], 'expected_arrival_time': expected_arrival_time, 'actual_arrival_time': actual_arrival_time, 'late': late, 'mins_late': mins_late})
                    break

                break
    time.sleep(60)

GTFS data downloaded and unzipped successfully.
Train has arrived
Train has arrived


In [67]:
# Define the URL and your API key
api_key = 'afec7635-a79b-4ccb-b87b-5c8d9cf5b36c'
operator = 'CT'
url = f'http://api.511.org/transit/datafeeds?api_key={api_key}&operator_id={operator}'

# # Download the GTFS zip file
# response = requests.get(url)
# response.raise_for_status()  # Ensure we got a successful response

# # Unzip the file
# zip_file = zipfile.ZipFile(io.BytesIO(response.content))
# zip_file.extractall('gtfs_data')  # Extract to a directory named 'gtfs_data'

# print("GTFS data downloaded and unzipped successfully.")
# stops_df = pd.read_csv('gtfs_data/stops.txt')
# # drop all non-numeric rows from stop_id
# stops_df = stops_df[stops_df['stop_id'].str.isnumeric()]

url = f'https://api.511.org/transit/VehicleMonitoring?api_key={api_key}&agency={operator}'
stop_data =[]
# Fetch the GTFS-RT data
response = requests.get(url)
response.raise_for_status()  #
# for each trip_id, get the next stop
data = response.content  # Your JSON data

# Decode the byte string to a regular string
data_str = data.decode('utf-8-sig')

# Parse the JSON data
json_data = json.loads(data_str)

# Extracting the relevant information
vehicle_activities = json_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']['VehicleActivity']

# get list of unique trip_ids
trip_ids = set()
for activity in vehicle_activities:
    trip_ids.add(activity['MonitoredVehicleJourney']['VehicleRef'])
for trip_id in trip_ids:
    for activity in vehicle_activities:
        journey = activity['MonitoredVehicleJourney']
        if journey['VehicleRef'] == trip_id:
            # print(f"trip_id: {journey['VehicleRef']}")
            vehicle_lat = float(journey['VehicleLocation']['Latitude'])
            vehicle_lon = float(journey['VehicleLocation']['Longitude'])
            # print(f"Vehicle Location: Lat {vehicle_lat}, Lon {vehicle_lon}")
            monitored_call = journey['MonitoredCall']
            stop_id = monitored_call['StopPointRef']
            timestamp = activity['RecordedAtTime']
            timestamp=datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z')  - timedelta(hours=7)
            timestamp = timestamp.replace(tzinfo=None)

            stop_data.append({'trip_id': trip_id, 'stop_id': stop_id, 'vehicle_lat':vehicle_lat,'vehicle_lon':vehicle_lon,'timestamp':timestamp})



In [68]:
stop_data

[{'trip_id': '249',
  'stop_id': '70211',
  'vehicle_lat': 37.3907394,
  'vehicle_lon': -122.066689,
  'timestamp': datetime.datetime(2024, 7, 7, 15, 32, 53)},
 {'trip_id': '248',
  'stop_id': '70092',
  'vehicle_lat': 37.5762482,
  'vehicle_lon': -122.335762,
  'timestamp': datetime.datetime(2024, 7, 7, 15, 32, 53)},
 {'trip_id': '252',
  'stop_id': '70012',
  'vehicle_lat': 37.7753296,
  'vehicle_lon': -122.396027,
  'timestamp': datetime.datetime(2024, 7, 7, 15, 32, 53)}]

In [66]:
trip_ids

{'248', '249', '252'}

In [71]:
timestamp.isoformat()

'2024-07-07T15:32:53'